# **Gradio Interface for Claude Citations**
This notebook demonstrates how to create a Gradio interface for using Claude's citation feature. The interface allows users to:

- Chat with Claude while referencing documents
- Citations within a collapsible bubble
- Choose between plain text and PDF documents
- Get citations for Claude's responses

## **Setup and Dependencies**
First, let's install and import all required libraries:

In [1]:
!pip install gradio anthropic -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.6/222.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.9 MB/s eta 0:00:00


In [2]:
import gradio as gr
import anthropic
import base64
from typing import List, Dict, Any

- `gradio`: For creating the web interface
- `anthropic`: To interact with Claude API
- `base64`: For encoding PDF files
- `typing`: For type hints


## **Default Document Configuration**

We set up a default document that will be used if no user content is provided:

In [6]:
DEFAULT_DOC = "The grass is pink and soil is green. The sky is red while the sun looks blue."

## **Utility Functions**

### **PDF File Handling**

In [3]:
def read_pdf_as_base64(file_path: str) -> str:
    """Read a PDF file and return its base64 encoded content."""
    with open(file_path, 'rb') as file:
        return base64.b64encode(file.read()).decode('utf-8')

This function:
- Takes a PDF file path as input
- Opens the file in binary mode
- Encodes it to base64
- Returns the encoded string

### **Message History Management**

In [7]:
def user_message(
    user_input: str,
    history: list,
    enable_citations: bool,
    doc_type: str,
    text_content: str,
    pdf_file: str,
    api_key: str
) -> tuple:

    history.append({
        "role": "user",
        "content": user_input,
    })
    return "", history

This function handles user messages:
- Logs message details for debugging
- Appends user message to chat history
- Returns empty string (to clear input) and updated history

### **Message Formatting**

In [8]:
def format_message_history(
    history: list,
    enable_citations: bool,
    doc_type: str,
    text_content: str,
    pdf_file: str
) -> List[Dict]:
    """Convert Gradio chat history to Anthropic message format."""
    formatted_messages = []

    # Add previous messages
    for msg in history[:-1]:
        if msg["role"] == "user":
            formatted_messages.append({
                "role": "user",
                "content": msg["content"]
            })
        elif msg["role"] == "assistant":
            if "metadata" not in msg or msg["metadata"] is None:
                formatted_messages.append({
                    "role": "assistant",
                    "content": msg["content"]
                })

    # Prepare the latest message
    latest_message = {
        "role": "user",
        "content": []
    }

    # Add document if citations are enabled
    if enable_citations:
        if doc_type == "plain_text":
            latest_message["content"].append({
                "type": "document",
                "source": {
                    "type": "text",
                    "media_type": "text/plain",
                    "data": text_content.strip() if text_content.strip() else DEFAULT_DOC
                },
                "title": "User Document" if text_content.strip() else "Sample Document",
                "citations": {"enabled": True}
            })
        elif doc_type == "pdf" and pdf_file:
            pdf_base64 = read_pdf_as_base64(pdf_file)
            latest_message["content"].append({
                "type": "document",
                "source": {
                    "type": "base64",
                    "media_type": "application/pdf",
                    "data": pdf_base64
                },
                "title": "User PDF Document",
                "citations": {"enabled": True}
            })

    # Add the user's question
    latest_message["content"].append({
        "type": "text",
        "text": history[-1]["content"]
    })

    formatted_messages.append(latest_message)
    return formatted_messages

This function:
1. Converts Gradio chat history to Anthropic's message format
2. Processes previous messages in history
3. Handles document preparation based on type:
   - For plain text: Creates text document source
   - For PDF: Creates base64-encoded document source
4. Adds user's question to the message
5. Returns formatted messages list

### **Bot Response Handler**

In [9]:
def bot_response(
    history: list,
    enable_citations: bool,
    doc_type: str,
    text_content: str,
    pdf_file: str,
    api_key: str
) -> List[Dict[str, Any]]:
    try:
        if not api_key:
            history.append({
                "role": "assistant",
                "content": "Please provide your Anthropic API key to continue."
            })
            return history

        # Initialize client with provided API key
        client = anthropic.Anthropic(api_key=api_key)

        messages = format_message_history(history, enable_citations, doc_type, text_content, pdf_file)

        response = client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=1024,
            messages=messages
        )

        # Initialize main response and citations
        main_response = ""
        citations = []

        # Process each content block
        for block in response.content:
            if block.type == "text":
                main_response += block.text
                if enable_citations and hasattr(block, 'citations') and block.citations:
                    for citation in block.citations:
                        if citation.cited_text not in citations:
                            citations.append(citation.cited_text)

        # Add main response
        history.append({
            "role": "assistant",
            "content": main_response
        })

        # Add citations if any were found and citations are enabled
        if enable_citations and citations:
            history.append({
                "role": "assistant",
                "content": "\n".join([f"• {cite}" for cite in citations]),
                "metadata": {"title": "📚 Citations"}
            })

        return history

    except Exception as e:
        error_message = str(e)
        if "401" in error_message:
            error_message = "Invalid API key. Please check your Anthropic API key and try again."
        history.append({
            "role": "assistant",
            "content": f"I apologize, but I encountered an error: {error_message}"
        })
        return history

This function:
1. Validates API key presence
2. Initializes Anthropic client
3. Formats message history
4. Makes API call to Claude
5. Processes response:
   - Extracts main response text
   - Collects citations if enabled
6. Updates chat history with:
   - Main response
   - Formatted citations (if any)
7. Handles errors:
   - API key validation
   - General error handling
   - Returns appropriate error messages


### **UI Component Management**

In [10]:
def update_document_inputs(enable_citations: bool, doc_type: str = "plain_text"):
    """Update visibility of document input components based on settings."""
    text_visible = enable_citations and doc_type == "plain_text"
    pdf_visible = enable_citations and doc_type == "pdf"
    radio_visible = enable_citations

    return {
        doc_type_radio: gr.Radio(visible=radio_visible),
        text_input: gr.Textbox(visible=text_visible),
        pdf_input: gr.File(visible=pdf_visible)
    }

This function:
- Controls visibility of UI components based on:
  - Citation toggle state
  - Selected document type
- Returns updated component visibility states

## **Gradio Interface Setup**

The interface is built using Gradio's `Blocks` API:

### **Component Configuration**

1. Chat Interface:
- Chatbot: Message display area
- Message Input: Text input for user messages

2. Settings Panel:
- API Key Input: Secure input for Anthropic API key
- Citations Toggle: Enable/disable citations
- Document Type Selection: Plain text or PDF
- Document Input: Text area or file upload based on type

### **Event Handlers**

1. Settings Updates
2. Message Submission

In [12]:
with gr.Blocks(theme="ocean", fill_height=True) as demo:
    gr.Markdown("# Chat with Citations")

    with gr.Row(scale=1):
        with gr.Column(scale=4):
            chatbot = gr.Chatbot(
                type="messages",
                bubble_full_width=False,
                show_label=False,
                scale=1
            )

            msg = gr.Textbox(
                placeholder="Enter your message here...",
                show_label=False,
                container=False
            )

        with gr.Column(scale=1):
            api_key = gr.Textbox(
                type="password",
                label="Anthropic API Key",
                placeholder="Enter your API key",
                info="Your API key will not be stored",
                interactive=True,
            )

            enable_citations = gr.Checkbox(
                label="Enable Citations",
                value=True,
                info="Toggle citation functionality"
            )

            doc_type_radio = gr.Radio(
                choices=["plain_text", "pdf"],
                value="plain_text",
                label="Document Type",
                info="Choose the type of document"
            )

            text_input = gr.Textbox(
                label="Document Content",
                placeholder=f"Enter your document text here. Default document is {DEFAULT_DOC}",
                lines=10,
                info="Enter the text you want to reference. If empty, default document will be used."
            )

            pdf_input = gr.File(
                label="Upload PDF",
                file_count="single",
                file_types=[".pdf"],
                type="filepath",
                visible=False
            )

    clear = gr.ClearButton([msg, chatbot, text_input, pdf_input])

    # Update input visibility based on settings
    enable_citations.change(
        update_document_inputs,
        inputs=[enable_citations, doc_type_radio],
        outputs=[doc_type_radio, text_input, pdf_input]
    )

    doc_type_radio.change(
        update_document_inputs,
        inputs=[enable_citations, doc_type_radio],
        outputs=[doc_type_radio, text_input, pdf_input]
    )

    # Handle message submission
    msg.submit(
        user_message,
        [msg, chatbot, enable_citations, doc_type_radio, text_input, pdf_input, api_key],
        [msg, chatbot],
        queue=False
    ).then(
        bot_response,
        [chatbot, enable_citations, doc_type_radio, text_input, pdf_input, api_key],
        chatbot
    )

if __name__ == "__main__":
    demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0c29a5a5a5fd5dd432.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0c29a5a5a5fd5dd432.gradio.live


## Key Features

1. **API Key Security**:
   - API key is handled securely using password field
   - Not stored between sessions
   - Validated before each request

2. **Document Handling**:
   - Supports both plain text and PDF documents
   - Dynamic UI updates based on document type
   - Default document fallback

3. **Citation Management**:
   - Toggle for citation functionality
   - Separates citations from main response
   - Formats citations for readability

4. **Error Handling**:
   - API key validation
   - Document processing errors
   - Clear error messages to users

5. **UI/UX Features**:
   - Clean, two-column layout
   - Dynamic component visibility
   - Clear button for resetting interface
   - Responsive design with proper scaling

## Usage Tips

1. Always enter your Anthropic API key first
2. Enable citations if you want references
3. Choose document type and provide content
4. Enter your question in the message box
5. Review both response and citations in chat

## Error Recovery

Common error scenarios and solutions:
1. Invalid API key: Re-enter correct key
2. PDF upload fails: Check file format and size
3. No response: Check network connection
4. Citation errors: Verify document content

This interface provides a user-friendly way to interact with Claude's citation feature while maintaining security and providing clear feedback to users.